In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')

In [3]:
from transformers import BertTokenizer
bert_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
 add_special_tokens=True,
 do_lower_case=False,
 max_length=150,
 pad_to_max_length=True,
 return_attention_mask=True,
 return_token_type_ids=True                                         
 )

In [4]:
data['label']=data['label'].replace({'FAKE':1, 'REAL': 0})

In [10]:
def bert_encoder(txt):
 encoded = tokenizer.encode_plus(txt, add_special_tokens=True, max_length=150, pad_to_max_length=True, return_attention_mask=True, return_token_type_ids=True)
 return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

In [ ]:
import tensorflow as tf
bert_train = [bert_encoder(text) for text in data['text']]

In [13]:
import numpy as np
bert_lbl = data['label']
bert_train = np.array(bert_train)
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=2)

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(bert_train, bert_lbl, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape)

(5068, 3, 150) (5068, 2)


In [16]:
tr_reviews, tr_segments, tr_masks = np.split(x_train, 3, axis=1)
val_reviews, val_segments, val_masks = np.split(x_val, 3, axis=1)

In [17]:
tr_reviews = tr_reviews.squeeze()
tr_segments = tr_segments.squeeze()
tr_masks = tr_masks.squeeze()
val_reviews = val_reviews.squeeze()
val_segments = val_segments.squeeze()
val_masks = val_masks.squeeze()

In [18]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
 return {"input_ids": input_ids,
 "attention_mask": attention_masks,
 "token_type_ids": token_type_ids},y

In [19]:
import tensorflow as tf
train_ds = tf.data.Dataset.from_tensor_slices((tr_reviews,tr_masks, tr_segments, y_train)).map(example_to_features).shuffle(100).batch(16)
valid_ds = tf.data.Dataset.from_tensor_slices((val_reviews,val_masks, val_segments, y_val)).map(example_to_features).shuffle(100).batch(16)

In [20]:
from transformers import TFBertForSequenceClassification
bert_model = TFBertForSequenceClassification.from_pretrained(bert_name)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import numpy as np

In [22]:
bert_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
bert_history = bert_model.fit(train_ds, epochs=5,validation_data=valid_ds)

Epoch 1/5
  3/317 [..............................] - ETA: 54:00 - loss: 6.2760 - accuracy: 0.3750 